In [1]:
import pandas as pd
import glob
import os
import re
import numpy as np

In [2]:
df = pd.read_csv("/home/smitesh22/Data/extent.csv")

In [3]:
filenames_DEM = set(df.filename)

In [4]:
filenames_GIS = glob.glob("/home/smitesh22/Data/GIS/*.zip")

In [5]:
filenames_GIS = {re.search(r'/([^/]+)\.zip$', name).group(1) for name in filenames_GIS}

In [6]:
assert filenames_GIS == filenames_DEM

In [7]:
from osgeo import ogr, osr

In [8]:
path = "/home/smitesh22/Data/GIS Extracted/"

In [9]:
directories = [os.path.join(path, name) for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]

In [10]:
driver = ogr.GetDriverByName('ESRI Shapefile')
file_paths = []
for directory in directories:
    for root, dirs, files in os.walk(directory):
        for dir_name in dirs:
            file_paths.append(os.path.join(root, dir_name))
            

In [11]:
driver = ogr.GetDriverByName('ESRI Shapefile')

for shape_file in file_paths[1::2]:
    match = re.findall(r'\d+', shape_file)
    match  = [int(num) for num in match[1:3]]
    dataset = driver.Open(shape_file, 0)
    layer = dataset.GetLayer()
    extent = layer.GetExtent()
    min_x, max_x, min_y, max_y = extent
    coords = [int(np.round(coord)) for coord in extent]
        
    assert abs(coords[2] - match[0]) in (0, 1) and abs(coords[0] - match[1]) in (0, 1), f'Error with file {shape_file}'
    
     